In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import textacy
%matplotlib inline

In [2]:
import mlp.preprocess as pre

mlp\preprocess.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["labels"][df["labels"] == missing_label] = -1  # to make a "non-category"
mlp\preprocess.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["labels"][df["labels"] > missing_label] = df["labels"][df["labels"] > missing_label].values - 1


In [3]:
from mlp.embed import TopicVectors, SemanticVectors
from mlp.predict import FilteredClassify
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import FunctionTransformer


In [5]:
corpus = pre.get_corpus()
df = pre.get_df()

# y = pre.get_labeled_data(df['labels'].values)
y = df['labels'].values

Corpus(3435 docs; 19565 tokens)


In [6]:
# sgd_w2v = Pipeline([
#     ('word2vec embedding', SemanticVectors()),
#     ('extract_labeled', FunctionTransformer(pre.get_labeled_data, validate=False)),  # extract labeled points
#     ('SGD Lin-SVC w/ElasticNet', SGDClassifier(class_weight='balanced',  # compensate for class freqs
#                                                penalty='elasticnet',   # L1 + L2 regularized
#                                                alpha=0.001,
#                                                n_iter=10))
# ])

sgd_w2v = Pipeline([
    ('word2vec embedding', SemanticVectors()),
    ('SGD Lin-SVC w/ElasticNet', FilteredClassify(SGDClassifier,
                                                  class_weight='balanced',  # compensate for class freqs
                                                  penalty='elasticnet',   # L1 + L2 regularized
                                                  alpha=0.001,
                                                  n_iter=10)
     )
])

sgd_w2v.fit(corpus, y)

print "score: {:2f}%".format((sgd_w2v.score(corpus, y))*100.)

score: 92.444444%


In [11]:
pipe = Pipeline([
    ('embedding', SemanticVectors()),
    ('classify', FilteredClassify(LinearSVC))
])


N_FEATURES_OPTIONS = [100, 200, 300, 400, 500]
C_OPTIONS = [1, 10, 100, 1000]

param_grid = [
    {
        'embedding': [TopicVectors()],
        'embedding__n_topics': N_FEATURES_OPTIONS,
        'embedding__model': ['lsa', 'nmf'],
        'classify__C': C_OPTIONS,
        'embedding__bow_kws': [{
                                'weighting': 'tfidf',  # change to tf if using LDA
                                'normalize': True,  # turn off if using LDA
                                'smooth_idf': True,  # turn off if using LDA
                               }]
    },
    {
        'embedding': [TopicVectors()],
        'embedding__n_topics': N_FEATURES_OPTIONS,
        'embedding__model': ['lda'],
        'classify__C': C_OPTIONS,
        'embedding__bow_kws': [{
                                'weighting': 'tf',  # change to tf if using LDA
                                'normalize': False,  # turn off if using LDA
                                'smooth_idf': False  # turn off if using LDA
                               }]
    },
    {
        'embedding': [SemanticVectors()],
        'classify__C': C_OPTIONS
    },
]
# reducer_labels = ['PCA', 'NMF', 'KBest(chi2)']

grid = GridSearchCV(pipe, cv=3, n_jobs=1, param_grid=param_grid)


In [12]:
grid.fit(corpus, y)

AttributeError: 'TopicVectors' object has no attribute 'get_params'

In [ ]:
pipe = Pipeline([
    ('embedding', SemanticVectors()),
    ('extract_labeled', FunctionTransformer(pre.get_labeled_data, validate=False)),  # extract labeled points
    ('classify', LinearSVC())
])

# SGDClassifier(class_weight='balanced',  # compensate for class freqs
#                                                penalty='elasticnet',   # L1 + L2 regularized
#                                                alpha=0.001,
#                                                n_iter=10)

N_FEATURES_OPTIONS = [100, 200, 300, 400, 500]
# C_OPTIONS = [1, 10, 100, 1000]
TOPIC_OPTIONS = ['lda', 'lsa', 'nmf']
param_grid = [
    {
        'embedding': [TopicVectors()],
        'embedding__n_topics': N_FEATURES_OPTIONS,
        'embedding__model': TOPIC_OPTIONS,
        'embedding__bow_kws': [{
                                'weighting': 'tfidf',  # change to tf if using LDA
                                'normalize': True,  # turn off if using LDA
                                'smooth_idf': True,  # turn off if using LDA
                               },
                                {
                                'weighting': 'tf',  # change to tf if using LDA
                                'normalize': False,  # turn off if using LDA
                                'smooth_idf': False,  # turn off if using LDA
                                }]
    },
    {
        'embedding': [SemanticVectors()]
    },
]
# reducer_labels = ['PCA', 'NMF', 'KBest(chi2)']

grid = GridSearchCV(pipe, cv=3, n_jobs=3, param_grid=param_grid)


In [18]:
grid.fit(corpus, y)

ValueError: Found input variables with inconsistent numbers of samples: [3435, 675]

In [20]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import numpy; print("NumPy", numpy.__version__)
import scipy; print("SciPy", scipy.__version__)
import sklearn; print("Scikit-Learn", sklearn.__version__)

Windows-7-6.1.7601-SP1
('Python', '2.7.13 |Anaconda custom (64-bit)| (default, Dec 19 2016, 13:29:36) [MSC v.1500 64 bit (AMD64)]')
('NumPy', '1.11.3')
('SciPy', '0.18.1')
('Scikit-Learn', '0.18.1')


In [7]:
[1,2,3][1:]

[2, 3]